### Transform data

In [2]:
import glob 
import xarray as xr
import pandas as pd
import geopandas as gpd
import duckdb
from shapely.geometry import Point

# --- Load ERA5 NetCDF data and prepare ---
nc_files = sorted(glob.glob("era5_monthly_data/*.nc"))

ds = xr.open_mfdataset(nc_files, combine="by_coords", chunks={})
climate_df = ds.to_dataframe().reset_index()

In [4]:
ds

<xarray.Dataset> Size: 139MB
Dimensions:     (valid_time: 384, latitude: 129, longitude: 117)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 3kB 2009-01-01 ... 2024-12-01T06:...
  * latitude    (latitude) float64 1kB 38.0 37.75 37.5 37.25 ... 6.5 6.25 6.0
  * longitude   (longitude) float64 936B 68.0 68.25 68.5 ... 96.5 96.75 97.0
    number      int64 8B 0
    expver      (valid_time) object 3kB dask.array<chunksize=(192,), meta=np.ndarray>
Data variables:
    tp          (valid_time, latitude, longitude) float32 23MB dask.array<chunksize=(192, 129, 117), meta=np.ndarray>
    ssrd        (valid_time, latitude, longitude) float32 23MB dask.array<chunksize=(192, 129, 117), meta=np.ndarray>
    t2m         (valid_time, latitude, longitude) float32 23MB dask.array<chunksize=(192, 129, 117), meta=np.ndarray>
    swvl1       (valid_time, latitude, longitude) float32 23MB dask.array<chunksize=(192, 129, 117), meta=np.ndarray>
    u10         (valid_time, latitude, longitude) float32 23MB dask.array<chunksize=(192, 129, 117), meta=np.ndarray>
    v10         (valid_time, latitude, longitude) float32 23MB dask.array<chunksize=(192, 129, 117), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-06-15T16:07 GRIB to CDM+CF via cfgrib-0.9.1...

In [ ]:
import glob 
import xarray as xr
import pandas as pd
import geopandas as gpd
import duckdb
from shapely.geometry import Point

# --- Load ERA5 NetCDF data and prepare ---
nc_files = sorted(glob.glob("era5_monthly_data/*.nc"))

ds = xr.open_mfdataset(nc_files, combine="by_coords", chunks={})
climate_df = ds.to_dataframe().reset_index()
climate_df["valid_time"] = pd.to_datetime(climate_df["date"].astype(str), format="%Y%m%d")

# --- Load ADM3 GPKG boundaries and convert to WKB ---
gdf = gpd.read_file("input/gadm41_IND.gpkg", layer="ADM_ADM_3").to_crs("EPSG:4326")
gdf["geometry"] = gdf.geometry.to_wkb()

# --- Connect to DuckDB (in-memory) and enable spatial extension ---
con = duckdb.connect(database=":memory:")
con.execute("INSTALL spatial; LOAD spatial;")

# --- Register DataFrames in DuckDB ---
con.register("climate_df", climate_df)
con.register("adm3_df", gdf)

# --- Perform spatial join (point-in-polygon using ST_Contains + ST_GeomFromWKB) ---
con.execute("""
    CREATE TABLE climate_with_region AS
    SELECT
        c.longitude, c.latitude, 
        a.GID_3, a.NAME_3, a.GID_2, a.NAME_2, a.GID_1, a.NAME_1
    FROM climate_df c
    LEFT JOIN adm3_df a
    ON ST_Contains(ST_GeomFromWKB(a.geometry), ST_Point(c.longitude, c.latitude))
    WHERE a.GID_3 IS NOT NULL;
""")

# --- Aggregate by ADM3 and date ---
agg_df = con.execute("""
    SELECT
        GID_3,
        NAME_3,
        date,
        AVG(t2m) - 273.15 AS avg_temperature,
        SUM(tp) AS total_precipitation,
        AVG(swvl1) AS avg_soil_moisture,
        AVG(u10) AS avg_wind_u,
        AVG(v10) AS avg_wind_v,
        AVG(ssrd) AS avg_solar_radiation
    FROM climate_with_region
    GROUP BY GID_3, NAME_3, date
    ORDER BY GID_3, date;
""").fetchdf()

# --- Create Climate Indicators Metadata Table ---
indicators_data = [
    {"GID": None, "NAME": None, "indicator": "temperature", "units": "Celsius", "description": "Average Temperature"},
    {"GID": None, "NAME": None, "indicator": "precipitation", "units": "mm", "description": "Total Precipitation"},
    {"GID": None, "NAME": None, "indicator": "soil_moisture", "units": "m³/m³", "description": "Average Soil Moisture"},
    {"GID": None, "NAME": None, "indicator": "wind_u", "units": "m/s", "description": "Average Wind U Component"},
    {"GID": None, "NAME": None, "indicator": "wind_v", "units": "m/s", "description": "Average Wind V Component"},
    {"GID": None, "NAME": None, "indicator": "solar_radiation", "units": "J/m²", "description": "Average Solar Radiation"}
]

# Create all combinations of GID/NAME with indicators
climate_indicators = []
unique_regions = agg_df[['GID_3', 'NAME_3']].drop_duplicates()

for _, region in unique_regions.iterrows():
    for indicator_info in indicators_data:
        climate_indicators.append({
            "GID": region['GID_3'],
            "NAME": region['NAME_3'],
            "indicator": indicator_info['indicator'],
            "units": indicator_info['units'],
            "ID": f"{region['GID_3']}_{indicator_info['indicator']}",
            "TITLE": f"{region['NAME_3']} - {indicator_info['description']}"
        })

climate_indicators_df = pd.DataFrame(climate_indicators)

# --- Create Time Series Values Table ---
time_series_data = []

for _, row in agg_df.iterrows():
    gid = row['GID_3']
    date = row['date']
    
    # Map column names to indicator names
    indicator_mapping = {
        'avg_temperature': 'temperature',
        'total_precipitation': 'precipitation', 
        'avg_soil_moisture': 'soil_moisture',
        'avg_wind_u': 'wind_u',
        'avg_wind_v': 'wind_v',
        'avg_solar_radiation': 'solar_radiation'
    }
    
    for col_name, indicator_name in indicator_mapping.items():
        series_id = f"{gid}_{indicator_name}"
        value = row[col_name]
        
        time_series_data.append({
            "series_id": series_id,
            "date": date,
            "value": value
        })

time_series_df = pd.DataFrame(time_series_data)

# --- Save both tables ---
climate_indicators_df.to_parquet("output/climate_indicators_adm3.parquet", index=False)
time_series_df.to_parquet("output/climate_timeseries_values.parquet", index=False)

print("✅ Saved climate indicators metadata to output/climate_indicators_adm3.parquet")
print("✅ Saved time series values to output/climate_timeseries_values.parquet")
print(f"📊 Created {len(climate_indicators_df)} indicator series for {len(unique_regions)} regions")
print(f"📈 Created {len(time_series_df)} time series data points")

KeyError: 'date'

### Check data

In [12]:
# --- Display structure of output files ---
import pandas as pd

# Load and display climate indicators table
print("=== CLIMATE INDICATORS TABLE (climate_indicators_adm3.parquet) ===")
indicators_df = pd.read_parquet("output/climate_indicators_adm3.parquet")
print(f"Shape: {indicators_df.shape}")
print("\nColumns:", list(indicators_df.columns))
print("\nSample data:")
print(indicators_df.head(10))
print("\nUnique indicators:", indicators_df['indicator'].unique())

print("\n" + "="*80)

# Load and display time series values table
print("=== TIME SERIES VALUES TABLE (climate_timeseries_values.parquet) ===")
values_df = pd.read_parquet("output/climate_timeseries_values.parquet")
print(f"Shape: {values_df.shape}")
print("\nColumns:", list(values_df.columns))
print("\nSample data:")
print(values_df.head(10))
print("\nDate range:", values_df['date'].min(), "to", values_df['date'].max())
print("\nUnique series count:", values_df['series_id'].nunique())

=== CLIMATE INDICATORS TABLE (climate_indicators_adm3.parquet) ===
Shape: (12048, 6)

Columns: ['GID', 'NAME', 'indicator', 'units', 'ID', 'TITLE']

Sample data:
           GID          NAME        indicator    units  \
0  IND.1.1.1_1  n.a. ( 2304)      temperature  Celsius   
1  IND.1.1.1_1  n.a. ( 2304)    precipitation       mm   
2  IND.1.1.1_1  n.a. ( 2304)    soil_moisture    m³/m³   
3  IND.1.1.1_1  n.a. ( 2304)           wind_u      m/s   
4  IND.1.1.1_1  n.a. ( 2304)           wind_v      m/s   
5  IND.1.1.1_1  n.a. ( 2304)  solar_radiation     J/m²   
6  IND.1.2.1_1  n.a. ( 2178)      temperature  Celsius   
7  IND.1.2.1_1  n.a. ( 2178)    precipitation       mm   
8  IND.1.2.1_1  n.a. ( 2178)    soil_moisture    m³/m³   
9  IND.1.2.1_1  n.a. ( 2178)           wind_u      m/s   

                            ID                                    TITLE  
0      IND.1.1.1_1_temperature       n.a. ( 2304) - Average Temperature  
1    IND.1.1.1_1_precipitation       n.a. ( 2304) -

In [10]:
gdf = gpd.read_file("input/gadm41_IND.gpkg", layer="ADM_ADM_3").to_crs("EPSG:4326")
gdf.geometry = gdf.geometry.to_wkb()

,GID_3,GID_0,COUNTRY,GID_1,NAME_1,NL_NAME_1,GID_2,NAME_2,NL_NAME_2,NAME_3,VARNAME_3,NL_NAME_3,TYPE_3,ENGTYPE_3,CC_3,HASC_3,geometry
0,IND.1.1.1_1,IND,India,IND.1_1,Andaman and Nicobar,NA,IND.1.1_1,Nicobar Islands,NA,n.a. ( 2304),NA,NA,Taluk,Taluk,NA,NA,"MULTIPOLYGON (((93.78988 6.85201, 93.79015 6.8..."
1,IND.1.2.1_1,IND,India,IND.1_1,Andaman and Nicobar,NA,IND.1.2_1,North and Middle Andaman,NA,n.a. ( 2178),NA,NA,Taluk,Taluk,NA,NA,"MULTIPOLYGON (((92.84441 12.14969, 92.84466 12..."
2,IND.1.3.1_1,IND,India,IND.1_1,Andaman and Nicobar,NA,IND.1.3_1,South Andaman,NA,n.a. ( 2178),NA,NA,Taluk,Taluk,NA,NA,"MULTIPOLYGON (((92.52111 10.89694, 92.52306 10..."
3,IND.2.1.1_1,IND,India,IND.2_1,Andhra Pradesh,NA,IND.2.1_1,Anantapur,NA,Anantapur,NA,NA,Taluk,Taluk,NA,NA,"MULTIPOLYGON (((77.84469 14.58576, 77.83913 14..."
4,IND.2.1.2_1,IND,India,IND.2_1,Andhra Pradesh,NA,IND.2.1_1,Anantapur,NA,Dharmavaram,NA,NA,Taluk,Taluk,NA,NA,"MULTIPOLYGON (((77.35452 14.52155, 77.35507 14..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2342,IND.36.19.1_1,IND,India,IND.36_1,West Bengal,NA,IND.36.19_1,South 24 Parganas,NA,Diamond Harbour,NA,NA,Taluk,Taluk,NA,NA,"MULTIPOLYGON (((88.02055 21.57222, 88.02139 21..."
2343,IND.36.19.2_1,IND,India,IND.36_1,West Bengal,NA,IND.36.19_1,South 24 Parganas,NA,n.a. ( 1187),NA,NA,Taluk,Taluk,NA,NA,"MULTIPOLYGON (((88.73339 22.0436, 88.73342 22...."
2344,IND.36.19.3_1,IND,India,IND.36_1,West Bengal,NA,IND.36.19_1,South 24 Parganas,NA,n.a. ( 1229),NA,NA,Taluk,Taluk,NA,NA,"MULTIPOLYGON (((88.49973 21.5975, 88.50222 21...."
2345,IND.36.20.1_1,IND,India,IND.36_1,West Bengal,NA,IND.36.20_1,Uttar Dinajpur,NA,Islampur,NA,NA,Taluk,Taluk,NA,NA,"MULTIPOLYGON (((88.3592 26.2426, 88.35914 26.2..."
